In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
dpevolucaoMensalCisp = pd.read_csv('rio-police-records/BaseDPEvolucaoMensalCisp.csv', encoding='latin', sep=';')

In [3]:
populacaoEvolucaoCispv = pd.read_csv('rio-police-records/PopulacaoEvolucaoMensalCisp.csv', encoding='latin', sep=';')

In [4]:
delegacias = pd.read_csv('rio-police-records/delegacias.csv', encoding='utf-8', sep=';')

In [73]:
dpevolucaoMensalCisp.columns

Index(['CISP', 'mes', 'vano', 'mes_ano', 'area_cisp', 'AISP', 'RISP', 'munic',
       'mcirc', 'Regiao', 'hom_doloso', 'lesao_corp_morte', 'latrocinio',
       'tentat_hom', 'lesao_corp_dolosa', 'estupro', 'hom_culposo',
       'lesao_corp_culposa', 'encontro_cadaver', 'encontro_ossada',
       'roubo_comercio', 'roubo_residencia', 'roubo_veiculo', 'roubo_carga',
       'roubo_transeunte', 'roubo_em_coletivo', 'roubo_banco',
       'roubo_cx_eletronico', 'roubo_celular', 'roubo_conducao_saque',
       'roubo_bicicleta', 'outros_roubos', 'total_roubos', 'furto_veiculos',
       'furto_bicicleta', 'outros_furtos', 'total_furtos', 'sequestro',
       'extorsao', 'sequestro_relampago', 'estelionato', 'apreensao_drogas',
       'recuperacao_veiculos', 'cump_mandado_prisao', 'ameaca',
       'pessoas_desaparecidas', 'hom_por_interv_policial', 'armas_apreendidas',
       'prisoes', 'grp', 'apf_cmp', 'apreensoes', 'gaai', 'aaapai_cmba',
       'registro_ocorrencias', 'pol_militares_mortos_serv

In [85]:
dpevolucaoMensalCisp[['vano']] = dpevolucaoMental[['vano']].astype(str)

In [ ]:
dpevolucaoMental

In [59]:
# plt.figure(figsize=(9,4))
# sns.countplot(dpevolucaoMental['vano'], palette='Dark2')

In [69]:
dpevolucaoMensalCisp['vano'].value_counts().sort_index()

2003    1526
2004    1560
2005    1560
2006    1560
2007    1560
2008    1567
2009    1572
2010    1586
2011    1632
2012    1632
2013    1634
2014    1656
2015    1656
2016    1656
2017     828
Name: vano, dtype: int64

In [70]:
dpevolucaoMensalCisp['Regiao'].value_counts()

Interior              8477
Capital               5192
Baixada Fluminense    3198
Interior              2814
Capital               1764
Grande Niterói        1320
Grande Niterói         420
Name: Regiao, dtype: int64

In [77]:
dpevolucaoMensalCisp['mes'].value_counts().sort_index()

1     1998
2     1998
3     1998
4     2000
5     2000
6     2001
7     1863
8     1865
9     1865
10    1865
11    1865
12    1867
Name: mes, dtype: int64

In [86]:
#Removido esta coluna por se tratar de uma coluna que contém dos mesmos dados da coluna "mes" e "vano"
dpevolucaoMensalCisp.drop(columns='mes_ano', inplace=True)

In [95]:
dpevolucaoMensalCisp['munic'] = dpevolucaoMensalCisp['munic'].apply(lambda x: x.strip())

In [107]:
#Existem registros que contém mais de um minicipio, por este motivo será utilizado o código de ibge para a identificação atráves da coluna "mcirc", excluindo a coluna 'munic'
len(dpevolucaoMensalCisp['munic'][dpevolucaoMensalCisp['munic'].apply(lambda x: ';' in x)])

1794

In [108]:
#Remoção da coluna 'munic'
dpevolucaoMensalCisp.drop(columns='munic', inplace=True)

In [125]:
#Porque a área da cisp está neste arquivo??
dpevolucaoMensalCisp['area_cisp'].describe()

count     23185
unique      145
top         3,5
freq        348
Name: area_cisp, dtype: object

In [133]:
dpevolucaoMensalCisp['grp'].describe()

count    4140.000000
mean       24.438889
std        23.500487
min         0.000000
25%         7.000000
50%        17.000000
75%        35.000000
max       261.000000
Name: grp, dtype: float64

In [136]:
dpevolucaoMensalCisp['apf_cmp'].describe()

count    4140.000000
mean       31.201208
std        31.343160
min         0.000000
25%         9.000000
50%        21.000000
75%        44.000000
max       287.000000
Name: apf_cmp, dtype: float64

In [149]:
dpevolucaoMensalCisp['fase'].value_counts()

3    22771
2      414
Name: fase, dtype: int64

In [187]:
csv = dpevolucaoMensalCisp.groupby('mcirc').sum().reset_index()[['mcirc','hom_doloso', 'lesao_corp_morte', 'latrocinio',
       'tentat_hom', 'lesao_corp_dolosa', 'estupro', 'hom_culposo',
       'lesao_corp_culposa', 'encontro_cadaver', 'encontro_ossada',
       'roubo_comercio', 'roubo_residencia', 'roubo_veiculo', 'roubo_carga',
       'roubo_transeunte', 'roubo_em_coletivo', 'roubo_banco',
       'roubo_cx_eletronico', 'roubo_celular']]

In [188]:
csv['mcirc'] = csv['mcirc'].apply(lambda x: str(x))

In [190]:
csv[['mcirc']].loc[0]

mcirc    3300100
Name: 0, dtype: object

In [189]:
csv[['mcirc']]

,mcirc
0,3300100
1,3300209
2,3300233
3,3300258
4,3300308
5,3300407
6,3300456
7,3300506
8,3300605
9,3300704


In [191]:
print(csv.to_csv())

,mcirc,hom_doloso,lesao_corp_morte,latrocinio,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,lesao_corp_culposa,encontro_cadaver,encontro_ossada,roubo_comercio,roubo_residencia,roubo_veiculo,roubo_carga,roubo_transeunte,roubo_em_coletivo,roubo_banco,roubo_cx_eletronico,roubo_celular
0,3300100,1021,3,11,1156,14313,770,480,5056,129,7,676,198,423,79,1195,414,2,3.0,239
1,3300209,708,1,23,418,8988,618,489,6672,41,4,396,323,748,132,1520,107,5,7.0,279
2,3300233,333,3,8,279,4433,228,121,2316,48,6,249,205,238,32,614,71,2,2.0,73
3,3300258,46,1,0,105,1343,104,43,371,19,1,20,8,19,4,106,2,0,0.0,18
4,3300308,223,5,6,323,8401,303,307,2421,81,7,130,69,93,75,190,69,2,0.0,59
5,3300407,583,7,9,808,9725,455,368,3588,51,4,911,119,439,169,1136,166,2,0.0,129
6,3300456,3346,10,41,1657,26418,1968,320,7501,134,10,1205,285,9145,1798,16219,1669,2,5.0,1995
7,3300506,54,1,2,80,1548,80,96,860,28,2,43,33,15,11,39,1,1,0.0,5
8,3300605,17,1,6,136,3474,165,78,754,46,0,24,20,16,7,53,2,0,0.0,11
9,3300704,790,5,12,809,622